<a href="https://colab.research.google.com/github/cow-coding/ML-DL-Study/blob/master/DL%20from%20Scratch/Book%201/Chap05/3.AffineSoftMaxLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library import

In [1]:
import numpy as np

# Affine Layer

순전파를 진행할 때는 행렬의 곱을 활용하였다.  
행렬 곱의 핵심은 차원의 원소 수 일치이다.  
이렇게 각 형상이 일치하는 두 행렬 곱 연산을 하는 것을 기하학적으로는 **어파인 변환 (affine transformation)**이라고 한다.

In [7]:
X = np.random.rand(2)
W = np.random.rand(2, 3)
B = np.random.rand(3)

print(f"Input shape : {X.shape}")
print(f"Weight shape : {W.shape}")
print(f"Bias shape : {B.shape}")

Y = np.dot(X, W) + B
print(f"Output shape : {Y.shape}")
print("\nY = X · W + B")

Input shape : (2,)
Weight shape : (2, 3)
Bias shape : (3,)
Output shape : (3,)

Y = X · W + B


## Batch Affine Layer

앞의 Affine layer는 1개의 X만 고려한 것이다.  
하지만 실제는 batch단위의 N개 데이터를 묶어서 입력으로 보낼 수 있다.  
배치 데이터를 처리할 수 있는 Affine layer를 구현해보자

In [13]:
X_dot_W = np.array([[0, 0, 0],
                   [10, 10, 10]])
B = np.array([1, 2, 3])

print(X_dot_W)
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [14]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
print(dY)

dB = np.sum(dY, axis=0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


예시는 N = 2인 batch size라고 가정했다.  
bias의 그래디언트는 두 데이터의 미분을 데이터마다 더해서 구한다.  

## Implementation Affine Layer

In [15]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

# Softmax with loss

softmax는 학습의 최종결과를 정규화하여 출력하는 역할을 한다.  
예를 들어 MNIST 데이터의 경우는 들어온 입력 값을 총 10개의 출력으로 정규화하여 반환한다.

In [16]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

주의할 점은 역전파 과정에서 batch_size로 나눠서 데이터 1개당 오차를 전파해야한다.